# Day 09. Exercise 03
# Ensembles

## 0. Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tqdm
import joblib
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, StackingClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import label_binarize

## 1. Preprocessing

1. Create the same dataframe as in the previous exercise.
2. Using `train_test_split` with parameters `test_size=0.2`, `random_state=21` get `X_train`, `y_train`, `X_test`, `y_test` and then get `X_train`, `y_train`, `X_valid`, `y_valid` from the previous `X_train`, `y_train`. Use the additional parameter `stratify`.

In [2]:
data = pd.read_csv('../data/day-of-week-not-scaled.csv')

In [3]:
weeks = pd.read_csv('../data/dayofweek.csv')
weeks = weeks['dayofweek']

In [4]:
X = data
y = weeks

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21, stratify=y)

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=21, stratify=y_train)

## 2. Individual classifiers

1. Train SVM, decision tree and random forest again with the best parameters that you got from the 01 exercise with `random_state=21` for all of them.
2. Evaluate `accuracy`, `precision`, and `recall` for them on the validation set.
3. The result of each cell of the section should look like this:

```
accuracy is 0.87778
precision is 0.88162
recall is 0.87778
```

In [7]:
svm = SVC(C=10, class_weight=None, gamma='auto', kernel='rbf', random_state=21, probability=True)
svm.fit(X_train, y_train)

y_pred = svm.predict(X_valid)

accuracy = accuracy_score(y_valid, y_pred)
precision = precision_score(y_valid, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_valid, y_pred, average='weighted', zero_division=0)

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')

accuracy is 0.87778
precision is 0.88162
recall is 0.87778


In [8]:
tree = DecisionTreeClassifier(class_weight='balanced', criterion='entropy', max_depth=25, random_state=21)
tree.fit(X_train, y_train)

y_pred = tree.predict(X_valid)

accuracy = accuracy_score(y_valid, y_pred)
precision = precision_score(y_valid, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_valid, y_pred, average='weighted', zero_division=0)

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')

accuracy is 0.85185
precision is 0.85481
recall is 0.85185


In [9]:
forest = RandomForestClassifier(class_weight='balanced', criterion='gini', max_depth=22, n_estimators=100, random_state=21)
forest.fit(X_train, y_train)

y_pred = forest.predict(X_valid)

accuracy = accuracy_score(y_valid, y_pred)
precision = precision_score(y_valid, y_pred, average='weighted', zero_division=0)
recall = recall_score(y_valid, y_pred, average='weighted', zero_division=0)

print(f'accuracy is {accuracy:.5f}')
print(f'precision is {precision:.5f}')
print(f'recall is {recall:.5f}')

accuracy is 0.90000
precision is 0.90208
recall is 0.90000


## 3. Voting classifiers

1. Using `VotingClassifier` and the three models that you have just trained, calculate the `accuracy`, `precision`, and `recall` on the validation set.
2. Play with the other parameteres.
3. Calculate the `accuracy`, `precision` and `recall` on the test set for the model with the best weights in terms of accuracy (if there are several of them with equal values, choose the one with the higher precision).

In [53]:
voting_clf = VotingClassifier(estimators=[('svm', svm), ('dt', tree), ('rf', forest)], voting='soft')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('svm',
                              SVC(C=10, gamma='auto', probability=True,
                                  random_state=21)),
                             ('dt',
                              DecisionTreeClassifier(class_weight='balanced',
                                                     criterion='entropy',
                                                     max_depth=25,
                                                     random_state=21)),
                             ('rf',
                              RandomForestClassifier(class_weight='balanced',
                                                     max_depth=22,
                                                     random_state=21))],
                 voting='soft')

In [54]:
y_pred_valid = voting_clf.predict(X_valid)

In [55]:
accuracy_valid = accuracy_score(y_valid, y_pred_valid)
precision_valid = precision_score(y_valid, y_pred_valid, average='weighted', zero_division=0)
recall_valid = recall_score(y_valid, y_pred_valid, average='weighted', zero_division=0)

print(f'accuracy is {accuracy_valid:.5f}')
print(f'precision is {precision_valid:.5f}')
print(f'recall is {recall_valid:.5f}')

accuracy is 0.88519
precision is 0.88709
recall is 0.88519


In [56]:
y_pred_test = voting_clf.predict(X_test)

In [57]:
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test, average='weighted', zero_division=0)
recall_test = recall_score(y_test, y_pred_test, average='weighted', zero_division=0)

print('Test:')
print(f'accuracy is {accuracy_test:.5f}')
print(f'precision is {precision_test:.5f}')
print(f'recall is {recall_test:.5f}')

Test:
accuracy is 0.89053
precision is 0.89215
recall is 0.89053


## 4. Bagging classifiers

1. Using `BaggingClassifier` and `SVM` with the best parameters create an ensemble, try different values of the `n_estimators`, use `random_state=21`.
2. Play with the other parameters.
3. Calculate the `accuracy`, `precision`, and `recall` for the model with the best parameters (in terms of accuracy) on the test set (if there are several of them with equal values, choose the one with the higher precision)

In [10]:
bagging_clf = BaggingClassifier(svm, n_estimators=50, random_state=21)
bagging_clf.fit(X_train, y_train)

BaggingClassifier(estimator=SVC(C=10, gamma='auto', probability=True,
                                random_state=21),
                  n_estimators=50, random_state=21)

In [11]:
y_pred_valid = bagging_clf.predict(X_valid)

accuracy_valid = accuracy_score(y_valid, y_pred_valid)
precision_valid = precision_score(y_valid, y_pred_valid, average='weighted', zero_division=0)
recall_valid = recall_score(y_valid, y_pred_valid, average='weighted', zero_division=0)

print('Valid:')
print(f'accuracy is {accuracy_valid:.5f}')
print(f'precision is {precision_valid:.5f}')
print(f'recall is {recall_valid:.5f}')

Valid:
accuracy is 0.88148
precision is 0.89035
recall is 0.88148


In [12]:
y_pred_test = bagging_clf.predict(X_test)

accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test, average='weighted', zero_division=0)
recall_test = recall_score(y_test, y_pred_test, average='weighted', zero_division=0)

print('Test:')
print(f'accuracy is {accuracy_test:.5f}')
print(f'precision is {precision_test:.5f}')
print(f'recall is {recall_test:.5f}')

Test:
accuracy is 0.88462
precision is 0.88941
recall is 0.88462


## 5. Stacking classifiers

1. To achieve reproducibility in this case you will have to create an object of cross-validation generator: `StratifiedKFold(n_splits=n, shuffle=True, random_state=21)`, where `n` you will try to optimize (the details are below).
2. Using `StackingClassifier` and the three models that you have recently trained, calculate the `accuracy`, `precision` and `recall` on the validation set, try different values of `n_splits` `[2, 3, 4, 5, 6, 7]` in the cross-validation generator and parameter `passthrough` in the classifier itself,
3. Calculate the `accuracy`, `precision`, and `recall` for the model with the best parameters (in terms of accuracy) on the test set (if there are several of them with equal values, choose the one with the higher precision). Use `final_estimator=LogisticRegression(solver='liblinear')`.

In [13]:

skf = StratifiedKFold(n_splits=2, shuffle=True, random_state=21)

stacking_clf = StackingClassifier(
        estimators=[('svm', svm), ('dt', tree), ('rf', forest)],
        final_estimator=LogisticRegression(solver='liblinear'),
        passthrough=True, 
        cv=skf
)

stacking_clf.fit(X_train, y_train)

y_pred_test = stacking_clf.predict(X_test)
    
accuracy_test = accuracy_score(y_test, y_pred_test)
precision_test = precision_score(y_test, y_pred_test, average='weighted', zero_division=0)
recall_test = recall_score(y_test, y_pred_test, average='weighted', zero_division=0)

print('Test:')
print(f'accuracy is {accuracy_test:.5f}')
print(f'precision is {precision_test:.5f}')
print(f'recall is {recall_test:.5f}')

Test:
accuracy is 0.92012
precision is 0.92091
recall is 0.92012


## 6. Predictions

1. Choose the best model in terms of accuracy (if there are several of them with equal values, choose the one with the higher precision).
2. Analyze: for which weekday your model makes the most errors (in % of the total number of samples of that class in your full dataset), for which labname and for which users.
3. Save the model.

In [14]:
error_analysis = {
    'weekday_errors': {},
    'labname_errors': {},
    'user_errors': {}
}

data['predictions'] = stacking_clf.predict(X)
data['target'] = weeks
data['errors'] = data['predictions'] != data['target']

In [15]:
for weekday in data['target'].unique():
    total_samples = data[data['target'] == weekday].shape[0]
    error_samples = data[(data['target'] == weekday) & (data['errors'])].shape[0]
    error_rate = (error_samples / total_samples) * 100 if total_samples > 0 else 0
    error_analysis['weekday_errors'][weekday] = error_rate

In [16]:
error_analysis['weekday_errors']

{np.int64(4): 2.8846153846153846,
 np.int64(5): 4.059040590405904,
 np.int64(6): 2.247191011235955,
 np.int64(0): 8.088235294117647,
 np.int64(1): 2.9197080291970803,
 np.int64(2): 4.026845637583892,
 np.int64(3): 2.0202020202020203}

In [17]:
for labname in data.columns[data.columns.str.startswith('labname_')]:
    total_samples = data[data[labname] == 1].shape[0]
    error_samples = data[(data[labname] == 1) & (data['errors'])].shape[0]
    error_rate = (error_samples / total_samples) * 100 if total_samples > 0 else 0
    error_analysis['labname_errors'][labname] = error_rate

In [18]:
error_analysis['labname_errors']

{'labname_code_rvw': 6.097560975609756,
 'labname_lab02': 0.0,
 'labname_lab03': 100.0,
 'labname_lab03s': 0.0,
 'labname_lab05s': 5.555555555555555,
 'labname_laba04': 5.617977528089887,
 'labname_laba04s': 7.6923076923076925,
 'labname_laba05': 0.9009009009009009,
 'labname_laba06': 6.25,
 'labname_laba06s': 9.836065573770492,
 'labname_project1': 1.8927444794952681}

In [19]:
for user_col in data.columns[data.columns.str.startswith('uid_user_')]:
    total_samples = data[data[user_col] == 1].shape[0]
    error_samples = data[(data[user_col] == 1) & (data['errors'])].shape[0]
    error_rate = (error_samples / total_samples) * 100 if total_samples > 0 else 0
    error_analysis['user_errors'][user_col] = error_rate

In [20]:
error_analysis['user_errors']

{'uid_user_0': 0.0,
 'uid_user_1': 2.1739130434782608,
 'uid_user_10': 1.4084507042253522,
 'uid_user_11': 0.0,
 'uid_user_12': 0.0,
 'uid_user_13': 1.6666666666666667,
 'uid_user_14': 5.303030303030303,
 'uid_user_15': 0.0,
 'uid_user_16': 6.25,
 'uid_user_17': 5.88235294117647,
 'uid_user_18': 5.714285714285714,
 'uid_user_19': 4.395604395604396,
 'uid_user_2': 4.958677685950414,
 'uid_user_20': 0.0,
 'uid_user_21': 4.545454545454546,
 'uid_user_22': 28.57142857142857,
 'uid_user_23': 25.0,
 'uid_user_24': 3.571428571428571,
 'uid_user_25': 2.5,
 'uid_user_26': 1.1111111111111112,
 'uid_user_27': 4.3478260869565215,
 'uid_user_28': 0.0,
 'uid_user_29': 6.25,
 'uid_user_3': 2.8169014084507045,
 'uid_user_30': 2.564102564102564,
 'uid_user_31': 4.0,
 'uid_user_4': 1.5957446808510638,
 'uid_user_6': 16.666666666666664,
 'uid_user_7': 0.0,
 'uid_user_8': 4.25531914893617}

In [21]:
joblib.dump(stacking_clf, '../data/stacking_clf_ex03.pkl')

['../data/stacking_clf_ex03.pkl']